In [1]:
!pip install torch torchvision torchaudio
!pip install ogb
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

wandb.login(key="Your-api-key")

# Learnable Root-Based BFS Positional Encoding GCN

In [3]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_undirected, degree
import wandb
from collections import deque

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = "GCN Graph classification(Protein)_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def prep_graph(d):
    d = d.clone()
    if d.edge_index.numel() > 0:
        d.edge_index = to_undirected(d.edge_index, num_nodes=d.num_nodes)
    if d.x is None:
        d.x = torch.ones((d.num_nodes, 1), dtype=torch.float)
    deg = degree(d.edge_index[0], d.num_nodes, dtype=torch.float) if d.edge_index.numel() > 0 else torch.zeros(d.num_nodes, dtype=torch.float)
    m = max(deg.max().item(), 1.0)
    deg = (deg / m).unsqueeze(1)
    d.x = torch.cat([d.x, deg], dim=1)
    return d

def bfs_rank_from_root(num_nodes, edge_index_local, root_local):
    adj = [[] for _ in range(num_nodes)]
    u, v = edge_index_local
    for i in range(u.numel()):
        a = int(u[i])
        b = int(v[i])
        if a != b:
            adj[a].append(b)
            adj[b].append(a)
    ranks = torch.full((num_nodes,), -1.0)
    if num_nodes == 0 or root_local < 0 or root_local >= num_nodes:
        return ranks
    q = deque([root_local])
    seen = [False] * num_nodes
    seen[root_local] = True
    order = []
    while q:
        cur = q.popleft()
        order.append(cur)
        for w in adj[cur]:
            if not seen[w]:
                seen[w] = True
                q.append(w)
    L = max(1, len(order))
    for i, node in enumerate(order):
        ranks[node] = float(i) / float(L)
    return ranks

class LearnableBFS_GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, bfs_dim=8, tau=1.0):
        super().__init__()
        self.tau = tau
        self.root_scorer = nn.Sequential(
            nn.Linear(in_channels, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
        self.bfs_proj = nn.Sequential(
            nn.Linear(1, 16),
            nn.ReLU(),
            nn.Linear(16, bfs_dim)
        )
        self.conv1 = GCNConv(in_channels + bfs_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, out_channels)

    @torch.no_grad()
    def _compute_bfs_rank_batch(self, x, edge_index, batch, root_logits):
        device = x.device
        N = x.size(0)
        ranks = torch.full((N,), -1.0, device='cpu')
        edge_index_cpu = edge_index.detach().cpu()
        batch_cpu = batch.detach().cpu()
        logits_cpu = root_logits.detach().cpu().squeeze(-1)
        num_graphs = int(batch_cpu.max().item()) + 1 if N > 0 else 0
        for g in range(num_graphs):
            idx = (batch_cpu == g).nonzero(as_tuple=False).view(-1)
            n = idx.numel()
            if n == 0:
                continue
            logits_g = logits_cpu[idx]
            probs_g = F.gumbel_softmax(logits_g, tau=self.tau, hard=True)
            root_local = int(probs_g.argmax().item())
            map_gl2lc = torch.full((N,), -1, dtype=torch.long)
            map_gl2lc[idx] = torch.arange(n, dtype=torch.long)
            u_all, v_all = edge_index_cpu
            m_u = map_gl2lc[u_all]
            m_v = map_gl2lc[v_all]
            mask = (m_u >= 0) & (m_v >= 0)
            u_loc = m_u[mask]
            v_loc = m_v[mask]
            ranks_g = bfs_rank_from_root(n, (u_loc, v_loc), root_local)
            ranks[idx] = ranks_g
        return ranks.to(device)

    def forward(self, x, edge_index, batch):
        root_logits = self.root_scorer(x)
        bfs_rank = self._compute_bfs_rank_batch(x, edge_index, batch, root_logits)
        bfs_feat = self.bfs_proj(bfs_rank.unsqueeze(1))
        x = torch.cat([x, bfs_feat], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def run_seed(seed):
    set_seed(seed)
    dataset = TUDataset(root='data/TUDataset', name='PROTEINS', transform=prep_graph)
    if dataset.num_features == 0:
        for data in dataset:
            data.x = torch.ones((data.num_nodes, 1))
    dataset = dataset.shuffle()
    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    in_channels = dataset[0].x.size(1)
    model = LearnableBFS_GCN(in_channels, 64, dataset.num_classes, bfs_dim=8, tau=1.0).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_LearnableBFS_STG_seed_{seed}",
        group="GCN_PROTEINS_LearnableBFS",
        job_type="learnable_bfs",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": "PROTEINS",
            "task": "Learnable BFS",
            "positional_encoding": "Learned root (ST-Gumbel) + BFS rank MLP",
            "seed": seed
        }
    )

    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        total_samples = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total_samples += data.num_graphs
        train_acc = correct / total_samples
        return total_loss / len(train_loader), train_acc

    def test(loader):
        model.eval()
        correct = 0
        total_samples = 0
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total_samples += data.num_graphs
        return correct / total_samples

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test(test_loader)
        print(f"Seed {seed} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


Processing...
Done!
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250820_184649-2j852ohz
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run GCN_LearnableBFS_STG_seed_1
wandb: ⭐️ View project at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal
wandb: 🚀 View run at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal/runs/2j852ohz


Seed 1 | Epoch 001 | Loss: 0.6736 | Train Acc: 0.6000 | Test Acc: 0.5785
Seed 1 | Epoch 002 | Loss: 0.6690 | Train Acc: 0.6090 | Test Acc: 0.5785
Seed 1 | Epoch 003 | Loss: 0.6663 | Train Acc: 0.6124 | Test Acc: 0.6099
Seed 1 | Epoch 004 | Loss: 0.6582 | Train Acc: 0.6112 | Test Acc: 0.6233
Seed 1 | Epoch 005 | Loss: 0.6557 | Train Acc: 0.6371 | Test Acc: 0.6233
Seed 1 | Epoch 006 | Loss: 0.6477 | Train Acc: 0.6382 | Test Acc: 0.6413
Seed 1 | Epoch 007 | Loss: 0.6412 | Train Acc: 0.6449 | Test Acc: 0.6368
Seed 1 | Epoch 008 | Loss: 0.6377 | Train Acc: 0.6461 | Test Acc: 0.6413
Seed 1 | Epoch 009 | Loss: 0.6303 | Train Acc: 0.6528 | Test Acc: 0.6143
Seed 1 | Epoch 010 | Loss: 0.6281 | Train Acc: 0.6640 | Test Acc: 0.6278
Seed 1 | Epoch 011 | Loss: 0.6227 | Train Acc: 0.6573 | Test Acc: 0.6368
Seed 1 | Epoch 012 | Loss: 0.6108 | Train Acc: 0.6697 | Test Acc: 0.6457
Seed 1 | Epoch 013 | Loss: 0.6131 | Train Acc: 0.6697 | Test Acc: 0.6547
Seed 1 | Epoch 014 | Loss: 0.6188 | Train Acc: 0.68

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
wandb:      loss ███▇▇▆▆▅▅▅▄▅▄▄▄▃▃▃▃▃▃▂▂▃▃▂▃▃▂▂▂▂▂▃▂▂▁▁▁▁
wandb:  test_acc ▁▁▃▄▄▅▅▃▄▅▆▆▆▆▆▆▆▇▆▆▆▆▆▆▆▆█▇▇▇▇▅█▆▇▆▇██▇
wandb: train_acc ▁▂▂▃▃▄▄▅▄▅▅▆▆▆▆▇▆▇▆▆▇▇▆▆▇▇▇▇▇▇▇▇▇▇█▇▇█▇█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.55783
wandb:  test_acc 0.67265
wandb: train_acc 0.71461
wandb: 
wandb: 🚀 View run GCN_LearnableBFS_STG_seed_1 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal/runs/2j852ohz
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250820_184649-2j852ohz/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/wor

Seed 2 | Epoch 001 | Loss: 0.6832 | Train Acc: 0.5596 | Test Acc: 0.6188
Seed 2 | Epoch 002 | Loss: 0.6737 | Train Acc: 0.5899 | Test Acc: 0.6188
Seed 2 | Epoch 003 | Loss: 0.6681 | Train Acc: 0.5899 | Test Acc: 0.6188
Seed 2 | Epoch 004 | Loss: 0.6643 | Train Acc: 0.6045 | Test Acc: 0.6816
Seed 2 | Epoch 005 | Loss: 0.6626 | Train Acc: 0.6303 | Test Acc: 0.6816
Seed 2 | Epoch 006 | Loss: 0.6610 | Train Acc: 0.6337 | Test Acc: 0.6413
Seed 2 | Epoch 007 | Loss: 0.6555 | Train Acc: 0.6416 | Test Acc: 0.6368
Seed 2 | Epoch 008 | Loss: 0.6324 | Train Acc: 0.6539 | Test Acc: 0.6996
Seed 2 | Epoch 009 | Loss: 0.6111 | Train Acc: 0.6843 | Test Acc: 0.7175
Seed 2 | Epoch 010 | Loss: 0.5846 | Train Acc: 0.7101 | Test Acc: 0.7534
Seed 2 | Epoch 011 | Loss: 0.5711 | Train Acc: 0.7101 | Test Acc: 0.7399
Seed 2 | Epoch 012 | Loss: 0.5604 | Train Acc: 0.7112 | Test Acc: 0.7444
Seed 2 | Epoch 013 | Loss: 0.5535 | Train Acc: 0.7382 | Test Acc: 0.7444
Seed 2 | Epoch 014 | Loss: 0.5527 | Train Acc: 0.72

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ██▇▇▇▇▆▅▄▄▃▃▃▂▃▂▂▃▃▃▃▂▂▂▂▁▂▂▂▂▂▂▂▂▂▁▁▂▁▂
wandb:  test_acc ▁▁▁▄▄▂▅▆▇▇▇█▇▇█▇█▆▇▇▇▇▇██▇█▇█▇▇▇▇▇▇▇▇▇▇▇
wandb: train_acc ▁▂▂▃▃▄▄▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇██▇█▇██▇▇███▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.52143
wandb:  test_acc 0.75785
wandb: train_acc 0.74157
wandb: 
wandb: 🚀 View run GCN_LearnableBFS_STG_seed_2 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal/runs/6bqdkj3c
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250820_184941-6bqdkj3c/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/wor

...

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ██▇▇▇▇▇▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▂▂▂▂▂▁▂▂▁▁▂▁▂▁▂▂▁▁
wandb:  test_acc ▁▃▃▃▂▃▃▄▂▃▅▅▄▆▆▆▆▅▄▆▇▇▆▇▇▇▇▇██▇▇█▇█████▇
wandb: train_acc ▁▂▂▃▂▃▃▄▄▄▅▆▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█████████
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.53354
wandb:  test_acc 0.78027
wandb: train_acc 0.73258
wandb: 
wandb: 🚀 View run GCN_LearnableBFS_STG_seed_28 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal/runs/90mw75fh
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250820_200227-90mw75fh/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/wo

Seed 29 | Epoch 001 | Loss: 0.6766 | Train Acc: 0.5955 | Test Acc: 0.5964
Seed 29 | Epoch 002 | Loss: 0.6708 | Train Acc: 0.5955 | Test Acc: 0.5964
Seed 29 | Epoch 003 | Loss: 0.6642 | Train Acc: 0.5955 | Test Acc: 0.5964
Seed 29 | Epoch 004 | Loss: 0.6587 | Train Acc: 0.6079 | Test Acc: 0.6413
Seed 29 | Epoch 005 | Loss: 0.6485 | Train Acc: 0.6371 | Test Acc: 0.6368
Seed 29 | Epoch 006 | Loss: 0.6359 | Train Acc: 0.6472 | Test Acc: 0.6457
Seed 29 | Epoch 007 | Loss: 0.6393 | Train Acc: 0.6427 | Test Acc: 0.6682
Seed 29 | Epoch 008 | Loss: 0.6179 | Train Acc: 0.6719 | Test Acc: 0.6726
Seed 29 | Epoch 009 | Loss: 0.6120 | Train Acc: 0.6787 | Test Acc: 0.6592
Seed 29 | Epoch 010 | Loss: 0.5997 | Train Acc: 0.6899 | Test Acc: 0.6547
Seed 29 | Epoch 011 | Loss: 0.5931 | Train Acc: 0.6831 | Test Acc: 0.6951
Seed 29 | Epoch 012 | Loss: 0.5999 | Train Acc: 0.7000 | Test Acc: 0.6996
Seed 29 | Epoch 013 | Loss: 0.5907 | Train Acc: 0.6944 | Test Acc: 0.6637
Seed 29 | Epoch 014 | Loss: 0.5852 | T

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ██▇▇▇▆▆▅▅▄▄▄▄▄▄▃▃▂▃▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁
wandb:  test_acc ▁▁▁▃▄▅▄▄▇▅▅▅▅▄▅▆▅▅▆▇▆▆▇▆▇▆▆▆▇█▅▇▇▆█▆▇▇▆▆
wandb: train_acc ▁▁▁▂▃▃▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▆▇▇▇▇▇▇▇▇█▇█▇▇█▇█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.51755
wandb:  test_acc 0.69058
wandb: train_acc 0.7573
wandb: 
wandb: 🚀 View run GCN_LearnableBFS_STG_seed_29 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal/runs/qov9z1z8
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250820_200513-qov9z1z8/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/wor

Seed 30 | Epoch 001 | Loss: 0.6821 | Train Acc: 0.5517 | Test Acc: 0.6592
Seed 30 | Epoch 002 | Loss: 0.6727 | Train Acc: 0.5854 | Test Acc: 0.6592
Seed 30 | Epoch 003 | Loss: 0.6664 | Train Acc: 0.6146 | Test Acc: 0.6682
Seed 30 | Epoch 004 | Loss: 0.6571 | Train Acc: 0.6461 | Test Acc: 0.6547
Seed 30 | Epoch 005 | Loss: 0.6446 | Train Acc: 0.6449 | Test Acc: 0.6771
Seed 30 | Epoch 006 | Loss: 0.6418 | Train Acc: 0.6438 | Test Acc: 0.6457
Seed 30 | Epoch 007 | Loss: 0.6304 | Train Acc: 0.6517 | Test Acc: 0.6413
Seed 30 | Epoch 008 | Loss: 0.6171 | Train Acc: 0.6708 | Test Acc: 0.6682
Seed 30 | Epoch 009 | Loss: 0.6034 | Train Acc: 0.7011 | Test Acc: 0.6861
Seed 30 | Epoch 010 | Loss: 0.5880 | Train Acc: 0.7000 | Test Acc: 0.6726
Seed 30 | Epoch 011 | Loss: 0.5797 | Train Acc: 0.6966 | Test Acc: 0.6906
Seed 30 | Epoch 012 | Loss: 0.5702 | Train Acc: 0.7157 | Test Acc: 0.6637
Seed 30 | Epoch 013 | Loss: 0.5645 | Train Acc: 0.7202 | Test Acc: 0.6726
Seed 30 | Epoch 014 | Loss: 0.5668 | T

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:      loss ██▇▇▆▆▅▅▄▄▃▃▃▃▂▂▂▃▂▂▂▂▂▂▂▁▂▁▁▂▂▁▁▁▂▁▁▁▁▁
wandb:  test_acc ▃▃▄▃▄▂▄▄▅▃▅▅▅▅▇▅▇▁▇▅▆▆▇█▇█▆▆███▆▇▆▆▇▇▇▆▇
wandb: train_acc ▁▂▃▄▄▅▆▆▆▇▆▇▇▇▇▇▇▇▇█▇▇▇▇▇▇███▇██▇███████
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.51431
wandb:  test_acc 0.713
wandb: train_acc 0.74831
wandb: 
wandb: 🚀 View run GCN_LearnableBFS_STG_seed_30 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal/runs/t1dgowhh
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250820_200802-t1dgowhh/logs


# **Layerwise BFS**

In [4]:
# **Layerwise BFS for REDDIT-BINARY**
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = "GCN Graph classification(Protein)_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, bfs_dim=8):
        super().__init__()
        self.bfs_proj = torch.nn.Sequential(
            torch.nn.Linear(1, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, bfs_dim)
        )
        self.conv1 = GCNConv(in_channels + bfs_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels + bfs_dim, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, bfs_rank, edge_index, batch):
        bfs_feat = self.bfs_proj(bfs_rank.unsqueeze(1))
        x = torch.cat([x, bfs_feat], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        bfs_feat2 = self.bfs_proj(bfs_rank.unsqueeze(1))
        x = torch.cat([x, bfs_feat2], dim=1)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def make_bfs_rank_transform():
    def transform(data):
        G = to_networkx(data, to_undirected=True)
        bfs_rank = torch.full((data.num_nodes,), -1.0, dtype=torch.float)
        if G.number_of_nodes() > 0:
            degrees = dict(G.degree())
            root = max(degrees, key=degrees.get) if degrees else 0
            bfs_order = list(nx.bfs_tree(G, root))
            L = max(1, len(bfs_order))
            for i, node in enumerate(bfs_order):
                bfs_rank[node] = i / L
        data.bfs_rank = bfs_rank
        # REDDIT-BINARY has no node features; use degree as feature or 1s
        if not hasattr(data, 'x') or data.x is None:
            deg = torch.tensor([d for _, d in G.degree()], dtype=torch.float).unsqueeze(1)
            data.x = deg
        return data
    return transform

def run_seed(SEED):
    set_seed(SEED)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dataset = TUDataset(root='data/TUDataset', name='PROTEINS', transform=make_bfs_rank_transform())
    torch.manual_seed(SEED)
    dataset = dataset.shuffle()
    train_dataset = dataset[:len(dataset)//2]
    test_dataset = dataset[len(dataset)//2:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_Layerwise_BFS_PROTEINS_seed_{SEED}",
        group="GCN_PROTEINS_Layerwise_BFS",
        job_type="layerwise-bfs",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": "PROTEINS",
            "task": "Layerwise_BFS_Injection",
            "seed": SEED,
            "cuda_available": torch.cuda.is_available(),
            "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
            "cuda_version": torch.version.cuda if torch.cuda.is_available() else None
        }
    )

    model = GCN(
        in_channels=dataset.num_features,
        hidden_channels=64,
        out_channels=dataset.num_classes
    ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.num_graphs
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        acc = correct / len(train_dataset)
        return total_loss / len(train_dataset), acc
    
    def test():
        model.eval()
        correct = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        return correct / len(test_dataset)
    
    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test()
        print(f"Seed {SEED} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})
    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250820_201049-uin5jswc
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run GCN_Layerwise_BFS_PROTEINS_seed_1
wandb: ⭐️ View project at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal
wandb: 🚀 View run at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal/runs/uin5jswc


Seed 1 | Epoch 001 | Loss: 0.6771 | Train Acc: 0.5953 | Test Acc: 0.5961
Seed 1 | Epoch 002 | Loss: 0.6789 | Train Acc: 0.5953 | Test Acc: 0.5961
Seed 1 | Epoch 003 | Loss: 0.6731 | Train Acc: 0.5953 | Test Acc: 0.5961
Seed 1 | Epoch 004 | Loss: 0.6681 | Train Acc: 0.5953 | Test Acc: 0.5961
Seed 1 | Epoch 005 | Loss: 0.6662 | Train Acc: 0.5953 | Test Acc: 0.6086
Seed 1 | Epoch 006 | Loss: 0.6655 | Train Acc: 0.6169 | Test Acc: 0.6140
Seed 1 | Epoch 007 | Loss: 0.6611 | Train Acc: 0.5971 | Test Acc: 0.6176
Seed 1 | Epoch 008 | Loss: 0.6557 | Train Acc: 0.6097 | Test Acc: 0.6463
Seed 1 | Epoch 009 | Loss: 0.6537 | Train Acc: 0.6313 | Test Acc: 0.6481
Seed 1 | Epoch 010 | Loss: 0.6443 | Train Acc: 0.6277 | Test Acc: 0.6517
Seed 1 | Epoch 011 | Loss: 0.6412 | Train Acc: 0.6565 | Test Acc: 0.6481
Seed 1 | Epoch 012 | Loss: 0.6231 | Train Acc: 0.6529 | Test Acc: 0.6661
Seed 1 | Epoch 013 | Loss: 0.6210 | Train Acc: 0.6781 | Test Acc: 0.6786
Seed 1 | Epoch 014 | Loss: 0.6048 | Train Acc: 0.70

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
wandb:      loss █████▇▇▇▇▆▅▅▄▄▃▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▂▁▁▁▁▁▁▂▂▁
wandb:  test_acc ▁▁▁▁▂▂▃▃▃▄▆▆▅▇▇▇▇▇▇▄██▇█▇▇█▇██▆██▇███▇██
wandb: train_acc ▁▁▁▁▁▁▂▂▂▃▄▅▆▆▆▇▇▇▇█▇▇▇█▇▇▇▇█▇▇▇█▇▇▇█▇▇▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.49506
wandb:  test_acc 0.73968
wandb: train_acc 0.75899
wandb: 
wandb: 🚀 View run GCN_Layerwise_BFS_PROTEINS_seed_1 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal/runs/uin5jswc
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250820_201049-uin5jswc/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kagg

Seed 2 | Epoch 001 | Loss: 0.6914 | Train Acc: 0.5252 | Test Acc: 0.5781
Seed 2 | Epoch 002 | Loss: 0.6680 | Train Acc: 0.6133 | Test Acc: 0.5781
Seed 2 | Epoch 003 | Loss: 0.6641 | Train Acc: 0.6133 | Test Acc: 0.5781
Seed 2 | Epoch 004 | Loss: 0.6629 | Train Acc: 0.6133 | Test Acc: 0.5781
Seed 2 | Epoch 005 | Loss: 0.6601 | Train Acc: 0.6133 | Test Acc: 0.5781
Seed 2 | Epoch 006 | Loss: 0.6557 | Train Acc: 0.6241 | Test Acc: 0.5978
Seed 2 | Epoch 007 | Loss: 0.6494 | Train Acc: 0.6295 | Test Acc: 0.5907
Seed 2 | Epoch 008 | Loss: 0.6440 | Train Acc: 0.6313 | Test Acc: 0.6050
Seed 2 | Epoch 009 | Loss: 0.6381 | Train Acc: 0.6385 | Test Acc: 0.6194
Seed 2 | Epoch 010 | Loss: 0.6291 | Train Acc: 0.6421 | Test Acc: 0.6140
Seed 2 | Epoch 011 | Loss: 0.6171 | Train Acc: 0.6691 | Test Acc: 0.6050
Seed 2 | Epoch 012 | Loss: 0.6029 | Train Acc: 0.6817 | Test Acc: 0.6661
Seed 2 | Epoch 013 | Loss: 0.5864 | Train Acc: 0.7248 | Test Acc: 0.6481
Seed 2 | Epoch 014 | Loss: 0.5683 | Train Acc: 0.71

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▇▇▇▇▆▆▅▅▄▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▂▁▁▁▁▁▂▂▁▁▁▁▁▁
wandb:  test_acc ▁▁▁▁▂▂▃▂▂▅▅▇▇▇▇▅▇▇█▇█▇▇█▇▇▇██▇▇█▇██▇▇█▇█
wandb: train_acc ▁▃▃▃▃▄▄▄▄▅▇▆▇▇▇██████████▇███████▇██████
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.50221
wandb:  test_acc 0.73968
wandb: train_acc 0.77338
wandb: 
wandb: 🚀 View run GCN_Layerwise_BFS_PROTEINS_seed_2 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal/runs/lqf7kmgq
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250820_201210-lqf7kmgq/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kagg

...

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
wandb:      loss █▇▇▇▇▆▆▆▅▅▅▅▄▄▄▄▄▃▄▄▄▃▄▃▃▃▂▃▂▂▂▂▂▂▂▁▂▁▂▁
wandb:  test_acc ▁▁▂▃▄▃▄▄▃▃▃▄▄▆▆▄▇▇▅▇▄▇▆▇██▆▆███▅█▇█▇▇▇▆▇
wandb: train_acc ▁▁▁▁▃▄▄▃▄▃▄▆▅▆▅▅▆▇▆▆▅▆▆▇▇█▆▆▇█▆▆▅▇▇█▇█▇█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.60649
wandb:  test_acc 0.69479
wandb: train_acc 0.70144
wandb: 
wandb: 🚀 View run GCN_Layerwise_BFS_PROTEINS_seed_28 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal/runs/eu6kx20v
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250820_204926-eu6kx20v/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kag

Seed 29 | Epoch 001 | Loss: 0.6811 | Train Acc: 0.5881 | Test Acc: 0.6086
Seed 29 | Epoch 002 | Loss: 0.6755 | Train Acc: 0.5935 | Test Acc: 0.6086
Seed 29 | Epoch 003 | Loss: 0.6710 | Train Acc: 0.5899 | Test Acc: 0.6212
Seed 29 | Epoch 004 | Loss: 0.6691 | Train Acc: 0.5953 | Test Acc: 0.6355
Seed 29 | Epoch 005 | Loss: 0.6673 | Train Acc: 0.6259 | Test Acc: 0.6355
Seed 29 | Epoch 006 | Loss: 0.6647 | Train Acc: 0.6061 | Test Acc: 0.6320
Seed 29 | Epoch 007 | Loss: 0.6631 | Train Acc: 0.6529 | Test Acc: 0.6320
Seed 29 | Epoch 008 | Loss: 0.6626 | Train Acc: 0.6313 | Test Acc: 0.6589
Seed 29 | Epoch 009 | Loss: 0.6588 | Train Acc: 0.6349 | Test Acc: 0.6589
Seed 29 | Epoch 010 | Loss: 0.6554 | Train Acc: 0.6565 | Test Acc: 0.6553
Seed 29 | Epoch 011 | Loss: 0.6505 | Train Acc: 0.6547 | Test Acc: 0.6517
Seed 29 | Epoch 012 | Loss: 0.6505 | Train Acc: 0.6547 | Test Acc: 0.6355
Seed 29 | Epoch 013 | Loss: 0.6427 | Train Acc: 0.6547 | Test Acc: 0.6445
Seed 29 | Epoch 014 | Loss: 0.6389 | T

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ███▇▇▇▇▇▇▇▆▆▆▅▅▄▃▃▂▃▂▂▃▂▁▂▁▂▁▂▁▂▂▂▁▁▂▁▂▁
wandb:  test_acc ▁▂▂▂▂▄▄▃▃▂▃▃▄▄▃▆▆▆▇▇▇▆▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇
wandb: train_acc ▁▁▁▂▂▃▃▄▃▃▄▄▅▅▆▇▇▇▇▇█▇▇▇█▇█▆█▇▇▇█▇██▇█▇█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.50574
wandb:  test_acc 0.7325
wandb: train_acc 0.76439
wandb: 
wandb: 🚀 View run GCN_Layerwise_BFS_PROTEINS_seed_29 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal/runs/qfohyjsc
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250820_205047-qfohyjsc/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kagg

Seed 30 | Epoch 001 | Loss: 0.6873 | Train Acc: 0.5504 | Test Acc: 0.6014
Seed 30 | Epoch 002 | Loss: 0.6776 | Train Acc: 0.5899 | Test Acc: 0.6014
Seed 30 | Epoch 003 | Loss: 0.6736 | Train Acc: 0.5899 | Test Acc: 0.6014
Seed 30 | Epoch 004 | Loss: 0.6716 | Train Acc: 0.5899 | Test Acc: 0.6014
Seed 30 | Epoch 005 | Loss: 0.6704 | Train Acc: 0.5899 | Test Acc: 0.6014
Seed 30 | Epoch 006 | Loss: 0.6659 | Train Acc: 0.5899 | Test Acc: 0.6014
Seed 30 | Epoch 007 | Loss: 0.6708 | Train Acc: 0.5953 | Test Acc: 0.6068
Seed 30 | Epoch 008 | Loss: 0.6627 | Train Acc: 0.6115 | Test Acc: 0.6373
Seed 30 | Epoch 009 | Loss: 0.6600 | Train Acc: 0.5917 | Test Acc: 0.6338
Seed 30 | Epoch 010 | Loss: 0.6546 | Train Acc: 0.6169 | Test Acc: 0.6373
Seed 30 | Epoch 011 | Loss: 0.6493 | Train Acc: 0.6079 | Test Acc: 0.6804
Seed 30 | Epoch 012 | Loss: 0.6500 | Train Acc: 0.6583 | Test Acc: 0.6373
Seed 30 | Epoch 013 | Loss: 0.6423 | Train Acc: 0.6565 | Test Acc: 0.6804
Seed 30 | Epoch 014 | Loss: 0.6363 | T

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ██▇▇▇▇▇▇▆▆▆▅▅▅▅▄▄▃▃▃▄▃▂▂▂▂▂▂▂▂▂▂▃▁▁▁▂▂▁▁
wandb:  test_acc ▁▁▁▁▁▁▃▂▃▄▄▄▄▅▃▅▅▆▆▅▄▆▆▇▆▇█▇▅▆▇▇▇▇███▇██
wandb: train_acc ▁▂▂▂▂▃▃▂▃▃▅▆▅▆▅▆▆▆▇▆▇▆▇▇▇▇█▇█▇▇▇▇▇▇▇██▇█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.55089
wandb:  test_acc 0.76661
wandb: train_acc 0.73921
wandb: 
wandb: 🚀 View run GCN_Layerwise_BFS_PROTEINS_seed_30 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal/runs/jtp6r3uy
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28Protein%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250820_205207-jtp6r3uy/logs


# **GCN Learnable BFS Sinusoidal**

In [5]:
def run_seed(SEED):
    set_seed(SEED)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = TUDataset(root="data/TUDataset", name="PROTEINS", transform=make_bfs_rank_transform())
    torch.manual_seed(SEED)
    dataset = dataset.shuffle()
    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)  # 80% for train
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_LearnableSinBFS_NoLayerwise_seed_{SEED}",
        group="GCN_PROTEINS_LearnableSinBFS_NoLayerwise",
        job_type="learnable-sin-bfs-nolayerwise",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": "PROTEINS",
            "task": "Learnable_Sinusoidal_BFS_NoLayerwise",
            "seed": SEED,
            "cuda_available": torch.cuda.is_available(),
            "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
            "cuda_version": torch.version.cuda if torch.cuda.is_available() else None
        }
    )

    model = GCN(
        in_channels=dataset.num_features,
        hidden_channels=64,
        out_channels=dataset.num_classes,
        bfs_dim=32,
        max_len=512
    ).to(device)

    opt = torch.optim.Adam(model.parameters(), lr=0.001)

    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        total = 0
        for data in train_loader:
            data = data.to(device)
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item() * data.num_graphs
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total += data.num_graphs
        return total_loss / len(train_dataset), correct / total  # return loss and train accuracy

    def test():
        model.eval()
        correct = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        return correct / len(test_dataset)

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test()
        print(f"Seed {SEED} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

    wandb.finish()


# **BFS and centrality**

In [6]:
def run_seed(SEED):
    set_seed(SEED)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = TUDataset(root="data/TUDataset", name="PROTEINS", transform=make_bfs_and_centrality_transform())
    torch.manual_seed(SEED)
    dataset = dataset.shuffle()
    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_BFS+Centrality_seed_{SEED}",
        group="GCN_PROTEINS_BFS_Centrality",
        job_type="bfs-centrality",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": "PROTEINS",
            "task": "BFS_plus_Centrality",
            "seed": SEED,
            "cuda_available": torch.cuda.is_available(),
            "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
            "cuda_version": torch.version.cuda if torch.cuda.is_available() else None
        }
    )

    model = GCN(
        in_channels=dataset.num_features if dataset.num_features else 1,
        hidden_channels=64,
        out_channels=dataset.num_classes,
        bfs_dim=32,
        cent_dim=16,
        max_len=512
    ).to(device)

    opt = torch.optim.Adam(model.parameters(), lr=0.001)

    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        total_nodes = 0
        for data in train_loader:
            data = data.to(device)
            out = model(data.x, data.bfs_rank, data.cent2, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item() * data.num_graphs
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total_nodes += data.num_graphs
        train_acc = correct / total_nodes
        return total_loss / len(train_dataset), train_acc

    def test():
        model.eval()
        correct = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.bfs_rank, data.cent2, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        return correct / len(test_dataset)

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test()
        print(f"Seed {SEED} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

    wandb.finish()
